### Import library

In [ ]:
""" Example 2 is inbalanced data set; ~2200 in PD and ~1100 in SNP
    Goal is to predict if mutation is SNP or PD
    CV branch
    
    Total samples: 3368
    2254 PD samples
    1111 SNP samples
"""

In [1]:
""" Imports the required libraries and packages
"""

import pandas as pd  #Import for data manipulation in dataframes
import numpy as np  #Array manipulation and calculates mean

import random as rd
import time

from sklearn.metrics import(
    matthews_corrcoef,  # MCC for evaluation
    balanced_accuracy_score, #hyperparameter evaluation
    f1_score,  #hyperparameter evaluation
    confusion_matrix,  # confusion matrix for classification evalutation
    classification_report #Return the F1, precision, and recall of a prediction
    )
from sklearn.model_selection import(
    train_test_split,  # Splits data frame into the training set and testing set
    GridSearchCV,  # Cross validation to improve hyperparameters
    RandomizedSearchCV,
    StratifiedKFold, # K-fold CV
    GroupKFold
        )
from sklearn.ensemble import RandomForestClassifier #SK learn API for classificastion random forests
from sklearn.tree import DecisionTreeClassifier #Single tree decisions 
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier #allows for confidence scores to be predicted for each
np.set_printoptions(threshold=np.inf, precision=3) #full array printing

### Generate random seed

In [ ]:
def Random_Seed(): #Generates a random seed
    random1 = rd.randrange(1,100) #Random integet between 1 and 100
    random2 =  time.time() #Time since UTC epoch
    Seed = int(random2//random1//1000)
    return Seed### Generate random seed

### Manual K-fold validation

In [19]:
df = pd.read_csv('E2.csv')
df_ordered = df.sort_values('pdbcode:chain:resnum:mutation') #Only column of PDB codes
df_ordered.to_csv('order.csv', index = False)

extract = []
for i in range(len(df_ordered)):
    split = df_ordered.iloc[:,0][i].partition(':')
    PDB = split[0]
    extract.append(PDB)
print(set(extract))



{'1ggl', '2e9x', '2z0e', '2y3i', '3k5k', '2pmf', '3omh', '1d8j', '2zoq', '2zw3', '2kll', '1jsg', '1saw', '2v5e', '1xdl', '3qik', '1eig', '2pzd', '3h95', '1z3u', '1zsy', '1pdg', '2jif', '2fy3', '1hzm', '1x0x', '2w2m', '1iur', '1ijq', '3nkx', '4a35', '1fuj', '2iag', '1imv', '2gzk', '2gyp', '2oq5', '2pid', '3pls', '1ivh', '2z7s', '2hw5', '1zbq', '2pm8', '3dwb', '3dal', '1mdu', '2vze', '2hi9', '1e51', '1hdr', '3apa', '2kw1', '2xwd', '2l3l', '2yuj', '2if1', '3cmy', '1r0d', '1bnk', '2xzz', '1wlp', '3pdf', '1fdh', '3gro', '1jei', '2gf9', '2fie', '2ra4', '3igu', '3szr', '2coe', '3n2z', '1bhg', '2iik', '2vns', '1bqq', '3qxm', '3f1s', '3bxw', '1cjy', '2l53', '2yhd', '2rfj', '2yvq', '3au4', '1esr', '1z7x', '3jui', '1fsu', '3ews', '3gfw', '3bhi', '1axn', '1tzs', '3lue', '1gr3', '1o1u', '3hy5', '1eak', '3e04', '2d8s', '3d7t', '2eax', '1fxy', '2yrb', '2ep8', '3fer', '3hy8', '3ifw', '2eel', '2anw', '3eqm', '4a04', '3nsi', '3qkj', '2k27', '1yb5', '3p08', '3d25', '3a1f', '3bg3', '1bqu', '2jxc', '3sh4',

### Clean dataset in pandas

In [ ]:
def Clean_data(file):
    """ Input:      file        The dataset to read

        Returns:    Input       Dataframe with of input features for training
                    Output      Dataframe of class labels for each instance in Input

        Create, clean and convert dataset E2.csv to PD dataframe. Drops uneeded columns, removes blank spaces, 
        and applies "One Hot Encoding" to convert classes (PD/SNP) to 1/0
    """

    df = pd.read_csv('E2.csv')

    #Remove unrequired column, replace blank spaces, reset index to run from 0
    df.drop(['pdbcode:chain:resnum:mutation'], axis=1, inplace=True)
    df.dropna() #drop rows with missing values
    df.replace(' ', '_', regex=True, inplace=True)
    df.reset_index(drop=True, inplace = True)

    Input = df.drop('dataset', axis =1)
    Output_encoded = pd.get_dummies(df, columns=['dataset']) #Encode the PD and SNP columns
    Output = Output_encoded['dataset_pd'].copy().astype('int32') #PD = 1, SNP = 0

    return Input, Output

### Split into training and testing, generate RF (whole dataset)

In [ ]:
def train(Input, Output, Seed):
    """ Input:      Input           Dataframe with of input features for training
                    Output          Dataframe of class labels for each instance in Input
                    Seed            Random Seed

        Returns:    Input_train     Features training data
                    Input_test      Features test data
                    Classes_train   Class label training data
                    Classes_test    Class label test data

        80% training and 20% testing split. Strartify ensures fixed poportion of labels are in both sets. 
        Random forest defined as RFC with x trees, random seed. Outputs the training data to files.
        """

    Input_train, Input_test, Classes_train, Classes_test = train_test_split(Input, Output, train_size = 0.8, random_state= Seed, stratify=Output) 
    Initial_RFC = RandomForestClassifier(random_state = Seed, verbose = 1)
    Initial_RFC.fit(Input_train, Classes_train)

    with open('Training Data.txt', 'w') as file: #Writes training features to text file
        file.write(Input_train.to_string())
    with open('Class labels.txt', 'w') as file: #Writes training class labels to text file
        file.write(Classes_train.to_string())
    with open('Test Data.txt', 'w') as file: #Writes testing features to text file
        file.write(Input_test.to_string())
    with open('Test labels.txt', 'w') as file: #Writes testing class labels to text file
        file.write(Classes_test.to_string())

    return Initial_RFC, Input_test, Classes_test, Input_train, Classes_train

### Initial evaluation

In [ ]:
def test(Initial_RFC, Input_test, Classes_test):
    """ Input:  Input_test      Features test data
                Classes_test    Class label test data

        Evaluates the training data before balancing. Random forest classifier makes prediction using the test features. True values 
        are the class labels testing data
    """

    Output_pred = Initial_RFC.predict(Input_test) #Always perdict on the unseen test data, as train has been used by the estimastor
    print(f"              **Initial Evaluation**\n")
    print(f"Confusion Matrix:\n {confusion_matrix(Classes_test, Output_pred)}")
    print(f"{classification_report(Classes_test, Output_pred)}\nMCC                {matthews_corrcoef(Classes_test, Output_pred)}")


### Balancing

In [ ]:
def find_minority_class(classData):
    """ Input:    classData  Array of class labels
        Returns:  minClass   The label for the minority class
                  minSize    The number of items in the minority class
                  maxSize    The number of items in the majority class

        Finds information about the inbalance in class sizes
    """
    
    Minority_count = 0
    Majority_count = 0
    for datum in classData:
        if datum == 1:
            Majority_count += 1
        elif datum == 0:
            Minority_count += 1

    minClass = 0
    minSize = Minority_count
    maxSize = Majority_count
    if Minority_count > Majority_count:
        minClass = 1
        minSize = Majority_count
        maxSize = Minority_count

    return minClass, minSize, maxSize

In [ ]:
def Balance_ratio(maxSize, minSize): 
    """ Input:      maxSize     The number of items in the majority class
                    minSize     The number of items in the minority class

        Returns:    BF          Number of balancing folds

        Calculate the number of balancing folds needed using ratio of majority to minority class size. Double to ensure sufficient
        majority class instances are sampled, then + 1 to make odd to allow weighted vote.
    """
    Divide = maxSize/minSize
    BF = (2 * round(Divide)) + 1 #Double ratio to nearest integer
    return BF

In [ ]:
def balance(inData, classData, minClass, minSize):
    """ Input:    inData          array of input data
                  classData       array of classes assigned
                  minorityClass   class label for the minority class
                  minoritySize    size of the minority class

        Returns: array of indexes that are of interest for a balanced dataset

        Perform the actual balancing between SNPs and PDs
    """
    usedLines = [False] * len(inData) #Array of false for length of data
    for i in range(len(inData)):
        if classData[i] == minClass:
            usedLines[i] = True
            
    usedCount = 0
    while usedCount < minSize:
        i = rd.randrange(len(inData))
        if usedLines[i] == False:
            usedCount += 1
            usedLines[i] = True       

    return usedLines

In [ ]:
def balance_data(inData, classData, usedLines):
    """     Input:     inData      array of input training data
                       classData   array of classes assigned to training data
                       usedLines   array of line indexes to print

            Returns:   Input_balance  Array of balanced input training data
                       Label_balance  Array of balanced classes assigned to training data
        Create arrays for the input training data and classes, as needed for predicting the probability.
        The index [i] is the identifier between the two arrays
    """
    Input_balance = []
    Label_balance = []
    for i in range(len(inData)):
        if usedLines[i]:
            Input_balance.append(inData[i])
            Label_balance.append(classData[i])
            
    Input_balance = np.stack(Input_balance, axis =0)
    Label_balance = np.stack(Label_balance, axis =0)
    
    return Input_balance, Label_balance

### Balance for n folds

In [ ]:
def Balance_Folds(BF, usedLines, Input_balance, Label_balance):
    """ Input:      BF                Number of balancing folds needed
                    usedLines         Array of line indexes to print
                    Input_balance     Input_balance  Array of balanced input training data
                    Label_balance     Array of balanced classes assigned to training data

        Returns:    Input_folds       List of 5 balanced arrays of training data
                    Output_folds      List of 5 balanced arrays of training data's labels

        Perform the balance_data() function n number of balancing fold times. Return lists for training data and labels
        where each item is the output of balance_data()
    """
    Input_folds = []
    Output_folds = []
    for fold in range(BF):
        Input_folds.append(Input_balance)
        Output_folds.append(Label_balance)


    with open('Balanced training data.txt', 'w') as f:
        for number, fold in zip(range(BF), Input_folds):
            f.write(f"Fold: {number}\n\n{fold}\n\n\n")
        
    return Input_folds, Output_folds

### RFC hyperparameter tuning

In [ ]:
 def Hyperparameter(BF, Input_folds, Output_folds):
    """ Input:      BF                Number of balancing folds needed
                    Input_folds       List of 5 balanced arrays of training data
                    Output_folds      List of 5 balanced arrays of training data's labels

        Returns:    BF_RFC_HP         List of optimized hyperparameters for each RFC

        Perform RandomSearchCV on each RFC to optimize number of trees, max depth and max samples
    """  
    estimator = RandomForestClassifier()
    param_grid = {
                'n_estimators':np.arange(50,500,50),
                'max_depth': np.arange(2, 10, 2),
                'max_samples': np.arange(0.2, 1.2, 0.2)
                  }
    BF_RFC_HP = []

    for i in range(BF):
        HPtuning = RandomizedSearchCV(
            estimator,
            param_grid, 
            scoring = 'balanced_accuracy',
            cv = 10,
            n_jobs = 6, #how many cores to run in parallel
            verbose = 2
            ).fit(Input_folds[i], Output_folds[i].ravel())
        BF_RFC_HP.append(HPtuning.best_params_)
    
    return(BF_RFC_HP)

### Train RFC on balanced dataset

In [ ]:
def BF_training(BF, Input_folds, Output_folds, BF_RFC_HP, Seed): 
    """ Input:      BF              Number of balancing folds
                    Input_folds     List of 5 balanced arrays for training data
                    Output_folds    List of 5 balanced arrays of training data's labels

        Returns:    BF_RFC          List of RFC's trained on data in each balancing fold

        Create a model that returns probability predictions for each fold, using Balance_Fold() as input
    """    
    BF_RFC = []
    
    for i in range(BF):
        BF_RFC.append(RandomForestClassifier(
                                             random_state = Seed,
                                             verbose = 1
                                            )) #Generates a RF for each fold 
        BF_RFC[i].fit(Input_folds[i], Output_folds[i].ravel()) #Fits the RFC to balanced training data    
        
    return BF_RFC

#### Test RFC on test set

In [ ]:
def BFC_test(BF_RFC, Input_test):
    """ Input:  BF_RFC          List of RFC's trained on data in each balancing fold
                Input_test      20% unseen testing data split before the balancing folds
                
        Returns:Prob_matrix     List of arrays. Each item is 2D matrix where the 1st dimension is each subset in balancing fold, 
                                2nd dimension is predicted probability
    
        Test the trained RFCs on the test set, then for every instance, outputs the predicted probability for each class
    """
    Prob_matrix = [] #Empty list
    Prob_matrixlist = []
    for i in range(len(BF_RFC)):
        Prob_list = BF_RFC[i].predict_proba(Input_test.values)
        Prob_matrix.append(Prob_list)   
        
    with open('Test probabilities.txt', 'w') as f:
        for number, line in zip(range(BF), Prob_matrix ):
            f.write(f"Fold: {number}\n\n   SNP    PD\n{line}\n\n\n")

    return Prob_matrix

### Weighted voting

In [ ]:
def Weighted_Vote(Prob_matrix, BF):
    """ Input:      Prob_matrix     List of arrays. 2D matrix where the 1st dimension is each subset in balancing fold, 
                                    2nd dimension is predicted probability
                    BF              Number of balancing folds

        Returns:    Final_vote      Weighted vote classification

        Calculate the final weighted vote using confidence scores (Sc). Binary classification formula Sc = 2|S0 - 0.5|
    """
    Sc_PD = [] #Empty list
    Sc_SNP = [] #Empty list
    for i in range(BF):
        Sc_PD.append(2* (Prob_matrix[i][:,1] - 0.5)) #Confidence scores for PD, for each fold
        Sc_SNP.append(2*(Prob_matrix[i][:,0] - 0.5)) #Confidence scores for SNP, for each fold
    

    Sum_PD = np.sum(Sc_PD, axis = 0) #Sum of all PD confidence scores. 1D Array
    Sum_SNP = np.sum(Sc_SNP, axis = 0) #Sum of all SNP confidence scores. 1D Array     

    Vote_arr = [] #Empty list
    
    for i in range(len(Classes_test)):
        if Sum_PD[i] >= Sum_SNP[i]:
            Vote_arr.append([1]) #Append PD classifications to list
        elif Sum_SNP[i] > Sum_PD[i]:
            Vote_arr.append([0]) #Append SNP classifications to list
            
    Final_vote = np.stack(Vote_arr) #Converts list of arrays to a 2D array, shape (674,1)
    Final_vote = Final_vote.ravel() #Flattens 2D array to 1D array
        
    return(Final_vote, Sum_PD, Sum_SNP) #Returns the final confidence scores


### Final confidence

In [ ]:
def Final_score(Sum_PD, Sum_SNP, BF):
    """ Input:      Sum_PD      Sum of confidence score for PD predictions
                    Sum_SNP     Sum of confidence score for SNP predictions

        Returns:    S_out        Final confidence score

        Calculate the final confidence score
    """
    
    S_Out = np.abs((Sum_PD - Sum_SNP) /(BF*2))
    np.savetxt('S_out.txt', S_Out, "%.3f")
    
    return S_Out


In [ ]:
def evalutation(Classes_test, Final_vote, S_Out):
    """ Input:      Classes_test       Class label test data
                    Final_vote         Weighted vote classification

        Evaluation metrics from RFC on test data with
    """
    Output_pred = Final_vote
    print(f"              ***Final Evaluation***\n")
    print(f"Confusion Matrix:\n {confusion_matrix(Classes_test, Output_pred)}")
    print(f"{classification_report(Classes_test, Output_pred)}\nMCC                {matthews_corrcoef(Classes_test, Output_pred)}")
    
    print(f"See file 'Classification.txt' for final classifications and confidence scores")
    np.savetxt('Classification.txt',
           np.column_stack([Final_vote, S_Out]),
           fmt = ["%.0f","%.3f"],
           delimiter ="      ",
           header = "Final classifications and confidence scores\n\n"
          )
    

### Main Program

In [ ]:
Seed                         = Random_Seed()

file                         = 'E2.csv'
Input, Output                = Clean_data(file)
RFC, Input_test, Classes_test, Input_train, Classes_train = train(Input, Output, Seed)
test(RFC,Input_test, Classes_test)

inData                       = pd.DataFrame(Input_train).to_numpy()
classData                    = pd.DataFrame(Classes_train).to_numpy()

minClass, minSize, maxSize   = find_minority_class(classData)
BF                           = Balance_ratio(maxSize, minSize)
usedLines                    = balance(inData, classData, minClass, minSize)

Input_balance, Label_balance = balance_data(inData, classData, usedLines)
Input_folds, Output_folds    = Balance_Folds(BF, usedLines, Input_balance, Label_balance)

BF_RFC_HP                    = Hyperparameter(BF, Input_folds, Output_folds)
BF_RFC                       = BF_training(BF, Input_folds, Output_folds, BF_RFC_HP, Seed)
Prob_matrix                  = BFC_test(BF_RFC, Input_test)

Final_vote, Sum_PD, Sum_SNP  = Weighted_Vote(Prob_matrix, BF)
S_Out                        = Final_score(Sum_PD, Sum_SNP, BF)

evalutation(Classes_test, Final_vote,S_Out)
